# Decision Tree Regression

참고: XGBoost와 사이킷런을 활용한 그레이디언트 부스팅 (한빛미디어, 2022)

* 결정 트리는 가지(branch) 분할을 통해 데이터를 두 개의 노드(node)로 나눔
* 가지 분할은 예측을 만드는 리프 노드(leaf node)까지 계속됨

### 1. 필요한 Python 라이브러리 Import 하기

In [ ]:
# 판다스와 넘파이를 임포트합니다.
import pandas as pd
import numpy as np

# train_test_split 함수를 임포트합니다.
from sklearn.model_selection import train_test_split

# DecisionTreeRegressor를 임포트합니다.
from sklearn.tree import DecisionTreeRegressor

# cross_val_score를 임포트합니다.
from sklearn.model_selection import cross_val_score

from sklearn import tree
import matplotlib.pyplot as plt

# 경고를 끕니다.
import warnings

warnings.filterwarnings("ignore")

### 2. 자전거 대여 데이터 읽기

In [ ]:
# bike_rentals_cleaned 데이터셋을 로드합니다.
df_bikes = pd.read_csv("bike_rentals_cleaned.csv")
df_bikes

In [ ]:
# describe()를 활용한 통계특성 확인하기
df_bikes.describe()

In [ ]:
# 데이터를 X와 y로 나눕니다.
X_bikes = df_bikes.iloc[:, :-1]
X_bikes

In [ ]:
y_bikes = df_bikes.iloc[:, -1]
y_bikes

### 3. DecisionTreeRegressor를 활용한 K-폴드 교차검증 시행

In [ ]:
# DecisionTreeRegressor 객체를 만듭니다.
reg = DecisionTreeRegressor(random_state=2)
reg

In [ ]:
# 평균 제곱 오차로 교차 검증 점수를 계산합니다. CV=5는 전체 데이터를 5개영역으로 구분하여 교차검증 시행
scores = cross_val_score(reg, X_bikes, y_bikes, scoring="neg_mean_squared_error", cv=5)
scores

### 4. K-폴드 교차검증의 결과 RMSE 산정

In [ ]:
# 제곱근을 계산합니다.
rmse = np.sqrt(-scores)
rmse

In [ ]:
# 5번 교차검증한 RMSE의 평균을 출력합니다.
print("RMSE 평균: %0.2f" % (rmse.mean()))

### 5. 모델 개발 및 성능 검증

In [ ]:
# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [ ]:
# DecisionTreeRegressor를 훈련 세트에서 훈련하고 점수를 계산합니다.
reg = DecisionTreeRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)

In [ ]:
fig = plt.figure(figsize=(16, 5))
ax = fig.add_subplot()
ax.scatter(
    X_train.index, y_train, label="Real Bike Rental Data", s=10, c="b", alpha=0.5
)
ax.scatter(
    X_train.index, y_pred, label="Predicted Bike Rental Data", s=10, c="r", alpha=0.5
)
ax.set_ylabel("Bike rental")
ax.grid()
fig.legend(loc="upper left", bbox_to_anchor=(0.13, 0.85))

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
ax.scatter(y_train, y_pred, s=10, c="b", alpha=0.5)
ax.set_xlabel("Real Bike Rental Data")
ax.set_ylabel("Predicted Bike Rental Data")
ax.grid()
fig.legend(loc="upper left", bbox_to_anchor=(0.13, 0.85))

In [ ]:
# DecisionTreeRegressor를 훈련 세트에서 훈련하고 점수를 계산합니다.
# 위의 그래프를 보면 모든 데이터 포인트를 완벽하게 맞추고 있음
# 위의 교차검증 결과 RMSE=1233.36과 비교시 과적합되었다고 볼 수 있음
from sklearn.metrics import mean_squared_error

reg_mse = mean_squared_error(y_train, y_pred)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

In [ ]:
# DecisionTreeRegressor의 가지분류에 대한 시각화
plt.figure(figsize=(30, 10))
tree.plot_tree(reg, proportion=True, max_depth=2)
plt.show()

### 6. 하이퍼파라미터 튜닝을 통한 과적합 해결: GridSearchCV

In [ ]:
# GridSearchCV를 임포트합니다.
from sklearn.model_selection import GridSearchCV

# max_depth 매개변수를 선택합니다.
params = {"max_depth": [None, 2, 3, 4, 6, 8, 10, 20]}

# 회귀 모델을 만듭니다.
reg = DecisionTreeRegressor(random_state=2)

# GridSearchCV 객체를 초기화합니다.
grid_reg = GridSearchCV(
    reg,
    params,
    scoring="neg_mean_squared_error",
    cv=5,
    return_train_score=True,
    n_jobs=-1,
)

# X_train와 y_train로 그리드 서치를 수행합니다.
grid_reg.fit(X_train, y_train)

# 최상의 매개변수를 추출합니다.
best_params = grid_reg.best_params_

# 최상의 매개변수를 출력합니다.
print("최상의 매개변수:", best_params)

In [ ]:
grid_reg.cv_results_["mean_test_score"]

In [ ]:
# 최상의 점수를 계산합니다.
best_score = np.sqrt(-grid_reg.best_score_)

# 최상의 점수를 출력합니다.
print("훈련 점수: {:.3f}".format(best_score))

In [ ]:
# 최상의 모델을 추출합니다.
best_model = grid_reg.best_estimator_

# 테스트 세트에서 예측을 만듭니다.
y_pred = best_model.predict(X_test)

# mean_squared_error를 임포트합니다.
from sklearn.metrics import mean_squared_error

# 테스트 세트의 제곱근 오차를 계산합니다.
rmse_test = mean_squared_error(y_test, y_pred) ** 0.5

# 테스트 세트 점수를 출력합니다.
print("테스트 점수: {:.3f}".format(rmse_test))

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()
ax.scatter(y_test, y_pred, s=15, c="b", alpha=0.5)
ax.set_xlabel("Real Bike Rental Data")
ax.set_ylabel("Predicted Bike Rental Data")
ax.set_xlim([0, 9000])
ax.set_ylim([0, 9000])
ax.grid()
fig.legend(loc="upper left", bbox_to_anchor=(0.13, 0.85))

### 7. 하이퍼파라미터 튜닝(min_samples_leaf)을 통한 과적합 해결: GridSearchCV

In [ ]:
# grid_search 함수를 만듭니다.
def grid_search(params, reg=DecisionTreeRegressor(random_state=2)):

    # GridSearchCV 객체를 만듭니다.
    grid_reg = GridSearchCV(
        reg, params, scoring="neg_mean_squared_error", cv=5, n_jobs=-1
    )

    # X_train와 y_train에서 그리드 서치를 수행합니다.
    grid_reg.fit(X_train, y_train)

    # 최상의 매개변수를 추출합니다.
    best_params = grid_reg.best_params_

    # 최상의 매개변수를 출력합니다.
    print("최상의 매개변수:", best_params)

    # 최상의 점수를 계산합니다.
    best_score = np.sqrt(-grid_reg.best_score_)

    # 최상의 점수를 출력합니다.
    print("훈련 점수: {:.3f}".format(best_score))

    # 테스트 세트에 대한 예측을 만듭니다.
    y_pred = grid_reg.predict(X_test)

    # 평균 제곱근 오차를 계산합니다.
    rmse_test = mean_squared_error(y_test, y_pred) ** 0.5

    # 테스트 세트 점수를 출력합니다.
    print("테스트 점수: {:.3f}".format(rmse_test))

In [ ]:
grid_search(params={"min_samples_leaf": [1, 2, 4, 6, 8, 10, 20, 30]})

In [ ]:
grid_search(
    params={
        "max_depth": [None, 2, 3, 4, 6, 8, 10, 20],
        "min_samples_leaf": [1, 2, 4, 6, 8, 10, 20, 30],
    }
)

In [ ]:
grid_search(params={"max_depth": [6, 7, 8, 9, 10], "min_samples_leaf": [3, 5, 7, 9]})